In [3]:
import os
print(os.getcwd())

/workspaces/Ic-Rna-2024/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy


In [ ]:
import pandas as pd
import numpy as np
import itertools
import os

 
diretorio = "/workspaces/Ic-Rna-2024/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy"


file_list = [os.path.join(diretorio, f) for f in os.listdir(diretorio) if f.endswith(".xlsx")]

def calcula_mse(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    df=pd.read_excel(f"1000_model_40_9_1.xlsx")
    Z =df['Z'].values.reshape(-1, 1)

    mse = np.mean((Z - Z_media) ** 2 )

    #print(f"mse: {mse}")
    return mse

def diversity(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    df=pd.read_excel(f"1000_model_40_9_1.xlsx")
    Z =df['Z'].values.reshape(-1, 1)

    
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    error = error1 + error2

    return np.mean(error.T)

def mse_var_bias_covar(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    df=pd.read_excel(f"1000_model_40_9_1.xlsx")
    Z =df['Z'].values.reshape(-1, 1)

    var1 = np.mean([(z_pred - Z_media) ** 2 for z_pred in z_preds])
    var = [np.mean((z_pred - Z_media) ** 2) for z_pred in z_preds]
    var = np.mean(np.array(var))

    print(f"var: {var}")
    print(f"var1: {var1}")
    print(var.shape)

    bias=np.mean((Z_media- Z) ** 2 )
    print(f"bias: {bias}")
    
    M = 10  #quantidade de redes
    cov_sum = 0
    for i in range(M):
        for j in range(M):
            if i != j:
                f_i = z_preds[i]
                f_j = z_preds[j]
                f_i_mean = np.mean(f_i)
                f_j_mean = np.mean(f_j)
                cov_ij = np.mean((f_i - f_i_mean) * (f_j - f_j_mean))
                cov_sum += cov_ij

    covar = cov_sum / (M * (M - 1))
    print(f"covar: {covar}")

    return ( var, bias, covar)


def filtrar_combinacoes(file_list, tamanho_conjunto, erro_max,mse_max,var_max,bias_max,covar_max):
    resultados = []
    
    for count, combinacao in enumerate(itertools.combinations(file_list, tamanho_conjunto), start=1):
        #print(f'Testando combinação {count}: {combinacao}')
        
        mse = calcula_mse(combinacao)
        erro = diversity(combinacao)
        var, bias, covar = mse_var_bias_covar(combinacao)
       

        if  (mse < mse_max):
            resultados.append({"Conjunto": combinacao, "MSE": mse, "Erro de Diversidade": erro, "var": var, "bias": bias, "covar": covar})
            print (f"mse: {mse}")
            salvar_resultados(resultados, pasta_destino1)
        
        if  (erro < erro_max):
            resultados.append({"Conjunto": combinacao, "MSE": mse, "Erro de Diversidade": erro, "var": var, "bias": bias, "covar": covar})
            print (f"ERRO: {erro}")
            salvar_resultados(resultados, pasta_destino)

        if  (var < var_max):
            resultados.append({"Conjunto": combinacao, "MSE": mse, "Erro de Diversidade": erro, "var": var, "bias": bias, "covar": covar})
            print (f"var: {var}")
            salvar_resultados(resultados, pasta_destino2)
        
        if  (bias < bias_max):
            resultados.append({"Conjunto": combinacao, "MSE": mse, "Erro de Diversidade": erro, "var": var, "bias": bias, "covar": covar})
            print (f"bias: {bias}")
            salvar_resultados(resultados, pasta_destino3)

        if  (covar < covar_max):
            resultados.append({"Conjunto": combinacao, "MSE": mse, "Erro de Diversidade": erro, "var": var, "bias": bias, "covar": covar})
            print (f"covar: {covar}")
            salvar_resultados(resultados, pasta_destino4)

    return resultados

def salvar_resultados(resultados, pasta_destino):
    
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)

    for count, resultado in enumerate(resultados, start=1):
        
        nome_arquivo = f"resultado_{count}.xlsx"
        caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)
        
        
        df = pd.DataFrame([resultado])
        
        
        df.to_excel(caminho_arquivo, index=False)
        print(f"Resultado salvo em: {caminho_arquivo}")


# Definição dos parâmetros
tamanho_conjunto = 10
erro_max = 0.6        # Limite do erro de diversidade
mse_max = 9           # Limite do mse_sup
var_max = 0.083       # Limite da variancia
covar_max = 4         # Limite da covariancia
bias_max = 0.69       # Limite do vies
pasta_destino = "resultados_filtrados_erro"
pasta_destino1 = "resultados_filtrados_mse"
pasta_destino2 = "resultados_filtrados_var"
pasta_destino3 = "resultados_filtrados_bias"
pasta_destino4 = "resultados_filtrados_covar"


# Execução do filtro
resultados_filtrados = filtrar_combinacoes(file_list, tamanho_conjunto, erro_max,mse_max,var_max,bias_max,covar_max)

var: 0.08373521276062869
var1: 0.08373521276062869
()
bias: 0.692097214907632
covar: 4.039324366438322
mse: 0.692097214907632
Resultado salvo em: resultados_filtrados_mse/resultado_1.xlsx
var: 0.08373521276062869
Resultado salvo em: resultados_filtrados_var/resultado_1.xlsx
Resultado salvo em: resultados_filtrados_var/resultado_2.xlsx
bias: 0.692097214907632
Resultado salvo em: resultados_filtrados_bias/resultado_1.xlsx
Resultado salvo em: resultados_filtrados_bias/resultado_2.xlsx
Resultado salvo em: resultados_filtrados_bias/resultado_3.xlsx
covar: 4.039324366438322
Resultado salvo em: resultados_filtrados_covar/resultado_1.xlsx
Resultado salvo em: resultados_filtrados_covar/resultado_2.xlsx
Resultado salvo em: resultados_filtrados_covar/resultado_3.xlsx
Resultado salvo em: resultados_filtrados_covar/resultado_4.xlsx
var: 0.0840903582738487
var1: 0.0840903582738487
()
bias: 0.6926455018516686
covar: 4.0364707856315505
mse: 0.6926455018516686
Resultado salvo em: resultados_filtrados_m

KeyboardInterrupt: 